# <div style="font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:200%; text-align:center;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0">Santa 2023 - Bidirectional Brute Force with Wildcards</div>
#### <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:150%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >TABLE OF CONTENTS<br><div>
* [IMPORTS](#1)
* [LOAD DATA](#2)
* [FUNCTIONS](#3)
* [SOLVE](#4)

Code modified from: https://www.kaggle.com/code/dinhttrandrise/bidirectional-brute-force-w-wildcards

<a id="1"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" > IMPORTS<br><div> 

In [1]:
import os
import time


import pandas as pd
from ast import literal_eval
from dataclasses import dataclass
import random
from sympy.combinatorics import Permutation
from typing import Dict, List
import zipfile
import numpy as np
import sqlite3

In [2]:
database_file = '../solutions.db'
solution_method = 'bidirectional brute force w/ wildcards'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

select_query = "SELECT id FROM solutions WHERE solution_method <> ? ORDER BY count"
    
# Execute the query
cursor.execute(select_query, (solution_method,))
unsolved_ids = cursor.fetchall()


# Commit the changes and close the connection
conn.commit()
conn.close()



RUN_TIME = 60 * 60 * 24 * 7
START_TIME = time.time()
TIMEOUT = RUN_TIME

INCLUDES = [id[0] for id in unsolved_ids]
INCLUDES


INCLUDES = list(range(333, 338))

EXCLUDES = []
MIN_SIZE = None
MAX_SIZE = None

DEBUG = False


<a id="2"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >LOAD DATA<br><div> 

In [3]:
with zipfile.ZipFile('../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col = 'id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

<a id="3"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >FUNCTIONS<br><div> 

In [4]:
def init_reverse_moves(moves):
    new_moves = {}
    
    for m in moves.keys():
        new_moves[m] = moves[m]
        xform = moves[m]
        m_inv = '-' + m
        xform_inv = len(xform) * [0]
        for i in range(len(xform)):
            xform_inv[xform[i]] = i
        new_moves[m_inv] = xform_inv

    return new_moves

def apply_move(move, state):

    m = move
    s = state.split(';')

    move_list = moves[m]
    new_state = []
    for i in move_list:
        new_state.append(s[i])
    s = new_state

    return ';'.join(s)

def reverse_move(move, state):
    m = move[1:] if move[0] == '-' else '-' + move
    return apply_move(m, state)

def expand(paths, reverse=False):
    states = list(paths.keys())
    for s in states:
        for m in moves:
            if reverse:
                next_s = reverse_move(m, s)
                if not next_s in paths:
                    paths[next_s] = paths[s] + [m]
            else:
                next_s = apply_move(m, s)
                if not next_s in paths:
                    paths[next_s] = paths[s] + [m]

In [5]:
def solve(pid):
    global moves, source_paths, dest_paths
    
    ddf = puzzles.loc[pid]
    puzzle_type = ddf['puzzle_type']
    solution_state = ddf['solution_state']
    initial_state = ddf['initial_state']
    num_wildcards = ddf['num_wildcards']

    idf = puzzle_info.loc[puzzle_type]
    allowed_moves = idf['allowed_moves']  
    moves = literal_eval(allowed_moves)

    moves = init_reverse_moves(moves)
    
    if num_wildcards < 2:    
        source_paths = {
            initial_state: []
        }
        
        dest_paths = {
            solution_state: []
        }

        start_time = time.time()
        solution = None
        count = 0
        while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
            count += 1

            if count % 2:
                expand(source_paths)
            else:
                expand(dest_paths, reverse=True)

            overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
            if len(overlap) > 0:
                mnsc = 10000000
                mnsl = None
                for oi in range(len(overlap)):
                    ol = overlap[oi]
                    sl = '.'.join(source_paths[ol] + list(reversed(dest_paths[ol])))
                    sz = len(sl.split('.'))
                    if sz < mnsc:
                        mnsc = sz
                        mnsl = sl
                solution = mnsl
                break
    else:
        ssl = solution_state.split(';')
        mn_score = 10000000
        mn_sol = None
        for i in range(len(ssl)):
            for j in range(len(ssl)):
                if j <= i:
                    continue
              
                sol = None
                
                ssln = ssl.copy()
                # swap two values in solution state. Could this be performed more deliberately?
                t = ssln[i]
                ssln[i] = ssln[j]
                ssln[j] = t
                ss = ';'.join(ssln)
                
                source_paths = {
                    initial_state: []
                }

                dest_paths = {
                    ss: []
                }

                start_time = time.time()
                solution = None
                count = 0
                while time.time() - START_TIME < RUN_TIME and time.time() - start_time < TIMEOUT:
                    count += 1

                    if count % 2:
                        expand(source_paths)
                    else:
                        expand(dest_paths, reverse=True)

                    overlap = list(set(source_paths.keys()).intersection(set(dest_paths.keys())))
                    if len(overlap) > 0:
                        mnsc = 10000000
                        mnsl = None
                        for oi in range(len(overlap)):
                            ol = overlap[oi]
                            sl = '.'.join(source_paths[ol] + list(reversed(dest_paths[ol])))
                            sz = len(sl.split('.'))
                            if sz < mnsc:
                                mnsc = sz
                                mnsl = sl
                                if DEBUG:
                                    print('=> [A] ' + str(sz) + ' : ' + sl)
                        sol = mnsl
                        break

                if sol is not None:
                    sz = len(sol.split('.'))
                    if sz < mn_score:
                        mn_score = sz
                        mn_sol = sol
                        
                        if DEBUG:
                            print('=> {' + str(sz) + '} ' + str(sol))
                        
        solution = mn_sol
        
    return solution

<a id="4"></a>
# <div style= "font-family: Cambria; font-weight:bold; letter-spacing: 0px; color:white; font-size:120%; text-align:left;padding:3.0px; background: #6A1B9A; border-bottom: 8px solid #9C27B0" >SOLVE<br><div> 

In [ ]:
moves = None
source_paths = None
dest_paths = None

database_file = '../solutions.db'
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()

grows = []
all_ids = submission['id'].unique()
for pid in INCLUDES:
    if INCLUDES is not None and pid not in INCLUDES:
        print('=> [' + str(pid) + '] Not included!')
        continue
    if EXCLUDES is not None and pid in EXCLUDES:
        print('=> [' + str(pid) + '] Excluded!')
        continue
        
    df = puzzles.loc[pid]
    solution_state = df['solution_state']
    size = len(solution_state.split(';'))
        
    if MIN_SIZE is not None and size <= MIN_SIZE:
        print('=> [' + str(pid) + '] Skipped by lower size!')
        continue

    if MAX_SIZE is not None and size >= MAX_SIZE:
        print('=> [' + str(pid) + '] Skipped by upper size!')
        continue

    solution = solve(pid)
    if solution is None:
        print('=> [' + str(pid) + '] Failed!')        
    else:
        print('=> [' + str(pid) + '] Success!')        
        
        grows.append({'id': pid, 'moves': solution})
        
        
        game_id = int(pid)
        moves = solution
        move_count = len(moves.split('.'))
        
        select_query = "SELECT count FROM solutions WHERE id = ?"
            
        # Execute the query
        cursor.execute(select_query, (game_id,))
        best_move_count = cursor.fetchone()
        
        if best_move_count[0] > move_count:
            print(f'Improvement to {game_id}')
            # Insert the moves into the database
            insert_query = "INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)"
            cursor.execute(insert_query, (game_id, moves, move_count, solution_method))
            conn.commit()
        

# Commit the changes and close the connection
conn.commit()
conn.close()